In [1]:
# loading the code
include("../src/DifferenceZeroTest.jl")


Welcome to Nemo version 0.18.2

Nemo comes with absolutely no warranty whatsoever



The Legendre duplication formula for the gamma function states that

$2^{z - 1}\Gamma(z)\Gamma(z + \frac{1}{2}) = \sqrt{\pi}\Gamma(2z)$

We will prove the formula using the difference equation satisfied by the asymptotic expansion of $\ln\Gamma(z)$ at inifinity:

$\ln\Gamma(z) \sim \frac{1}{2}\ln(2\pi) + (z - 1/2)\ln z - z + S(\frac{1}{z}),$

where $S(\frac{1}{z})$ is a power series. The difference equation for the gamma function implies the following difference equation for $S(z)$:

$z (\sigma(S(z)) - S(z)) - z + (1 + z/2) \ln(1 + z) = 0,$

where $\sigma(S(z)) = S\left( \frac{z}{1 + z} \right)$.

In terms of the function $S(z)$ the Legendre duplication formula can be written as:

$z \left(S\left(\frac{z}{2}\right) - S(z) - S\left(\frac{z}{1 + z/2}\right)\right) - \ln(1 + z/2) + \frac{z}{2} = 0$

To prove this identity, we will construct an effective power series domain containing all the necessary functions. 
*On y va!*

In [2]:
# defining the shift operator
Z = IdentityPS{fmpq}(Nemo.QQ)
S = Z / (1 + Z);

Adjoin $z$

In [3]:
polyring, (x0, x1) = PolynomialRing(Nemo.QQ, ["x0", "x1"])
z = AnnihilatorBasedPS{fmpq}(x1 * (1 + x0) - x0, S, [0, 1, 0, 0])
D = construct_domain(z)
z = gen(D);

Then we adjoin all the necessary logarithms:
* $\ln(1 + z)$ which is $\sigma$-transcendental over $\mathbb{C}(z)$
* $\ln\left(1 + \frac{z}{1 + z/2}\right)$ which is $\sigma$-transcendental over $\mathbb{C}(z, \ln(1 + z), \sigma(\ln(1 + z)), \ldots)$
* $\ln(1 + z/2)$ satisfying a $\sigma$-equation over the field generated by already adjoined logarithms

In [4]:
# log(1 + z)
D = adjoin_transcendental(D, LogPS{fmpq}(Nemo.QQ))
logz = gen(D)

# adjoining log(1 + z/(1 + z/2))
D = adjoin_transcendental(D, LogPS{fmpq}(Nemo.QQ)(Z / (1 + 1//2 * Z)))
logz_plus_half = gen(D)

# adjoining log(1 + z/2)
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = x1 - x0 - logz_plus_half + logz
logz_half = AnnihilatorBasedPS{fmpq}(annihilator, S, [0, 1//2, -1//8, 1//24])
D = adjoin_sigma_algebraic(D, logz_half)
logz_half = gen(D);

Adjoin $S(z)$ (defined by the annihilator above and initial terms)

In [5]:
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = z * (x1 - x0) - z + (1 + z) * logz - 1//2 * z * logz
stirling = AnnihilatorBasedPS{fmpq}(annihilator, S, [0, 1//12, 0, -1//360, 0, 1//1260, 0])
D = adjoin_sigma_algebraic(D, stirling)
stirling = gen(D);

Adjoining $S(z / 2)$ and $S\left(\frac{z}{1 + z/2}\right)$ in a similar way

In [6]:
# S(z / (1 + z/2))
ic = [coeff(get_truncation(stirling(Z / (1 + Z * 1//2)), 5), i) for i in 0:4]
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = z * (x1 - x0) - z + (1 + z * 3//2) * logz_plus_half - 1//2 * z * logz_plus_half
stirling_plus_half = AnnihilatorBasedPS{fmpq}(annihilator, S, ic)
D = adjoin_sigma_algebraic(D, stirling_plus_half)
stirling_plus_half = gen(D)

# S(z / 2)
ic = [coeff(get_truncation(stirling(Z * 1//2), 5), i) for i in 0:4]
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = z * (x1 - x0) + (2 + 1//2 * z) * logz + z * logz - z * logz_half - 2 * z
stirling_half = AnnihilatorBasedPS{fmpq}(annihilator, S, ic)
D = adjoin_sigma_algebraic(D, stirling_half)
stirling_half = gen(D);

The expression to test:

In [7]:
legendre = z * (stirling_half - stirling - stirling_plus_half) - logz_half + 1//2 * z;

Test!

In [8]:
iszero(legendre)

true